# Sample from Slater determinants

A Slater determinant is a state obtained by applying an [orbital rotation](../explanations/orbital-rotation.ipynb) to an electronic configuration. The [ffsim.sample_slater](../api/ffsim.rst#ffsim.sample_slater) function draws electronic configuration samples from a Slater determinant specified by an orbital rotation and reference configuration. Unlike [ffsim.sample_state_vector](../api/ffsim.rst#ffsim.sample_state_vector), which requires the full state vector as input, `sample_slater` implements an efficient algorithm that works directly with the orbital rotation representation, allowing it to scale to much larger systems. This guide demonstrates the use of `sample_slater` on a few example systems.

## Hartree-Fock state

The simplest Slater determinant is the Hartree-Fock state, in which the lowest $N_\alpha$ spin-up and $N_\beta$ spin-down orbitals are occupied, and no orbital rotation is applied. Since all probability is concentrated on a single configuration, every shot returns the same bitstring.

In [1]:
import numpy as np

import ffsim

rng = np.random.default_rng(12345)

norb = 10
n_alpha = 5
n_beta = 3

samples = ffsim.sample_slater(norb, (range(n_alpha), range(n_beta)), shots=5, seed=rng)
samples

['00000001110000011111',
 '00000001110000011111',
 '00000001110000011111',
 '00000001110000011111',
 '00000001110000011111']

## Random Slater determinant

The following code cell samples a random Slater determinant with a few hundred orbitals and electrons.

In [2]:
norb = 500
n_alpha = 100
n_beta = 100

orbital_rotation = ffsim.random.random_unitary(norb, seed=rng)
occupied_orbitals = (range(n_alpha), range(n_beta))

samples = ffsim.sample_slater(
    norb, occupied_orbitals, orbital_rotation, shots=10, seed=rng
)
samples

['00000000001100000110000000100000010000110011000001010001001110010110000000000001010000000000010000000000000000000100010000000000101001000000000000010001100010000001000001000010000010010010100000010000000000001100001001001000001111001010010110000101010000000000001001000000011000001001000001100000101001000000000001001000000010000100010000010100001000001011001000000010000000000000000000101010001000000000000000000000010000010000000000000000100000000000001000010000010011110000000001110000101000001000110000100010000000000000100010000000110000000000000100100000001000000000010010010000000000000001001110010001000000100000000010000010100010100000011100000000010000000000010000010001010110000000000010001000000000001000000001000100000000000010000001000000001100000000100001000000101000011000000110001101000000011000000010000000010001000011001001000010000000100100100000011000000000000000000000010000001001000110100010100100000000000101001001100110101000001010110000000000110101000001010001000000000000

## Tight-binding model

Our last example is the ground state of a tight-binding model of spinless fermions on a one-dimensional chain, at half filling. The Hamiltonian is given by

$$
H = -t \sum_{j=0}^{N-2} \left(a_j^\dagger a_{j+1} + a_{j+1}^\dagger a_j\right).
$$

The Hamiltonian is diagonal in a basis of single-particle orbitals, and the ground state is constructed by filling the lowest-energy orbitals.

In [3]:
from collections import Counter

norb = 20
nelec = 10  # half-filling

# Construct the matrix encoding the Hamiltonian
tunneling = 1.0
mat = np.diag(-tunneling * np.ones(norb - 1), k=1) + np.diag(
    -tunneling * np.ones(norb - 1), k=-1
)

# Columns of orbital_rotation are the single-particle orbitals, sorted by energy
energies, orbital_rotation = np.linalg.eigh(mat)
# Fill the lowest-energy orbitals
occupied_orbitals = list(range(nelec))
print(f"Ground state energy: {sum(energies[occupied_orbitals])}")

# Sample electronic configurations
samples = Counter(
    ffsim.sample_slater(
        norb, occupied_orbitals, orbital_rotation, shots=10_000, seed=rng
    )
)
samples.most_common(10)

Ground state energy: -12.381489999654752


[('10101001010101101010', 12),
 ('01010101010101011010', 11),
 ('01010101011010101010', 11),
 ('10101010101010010101', 11),
 ('10101010101010101010', 10),
 ('01010101011001010101', 10),
 ('01101010101010101001', 9),
 ('10101010101010100101', 9),
 ('01011010101010101010', 8),
 ('10101010100101010110', 8)]